<a href="https://colab.research.google.com/github/cyberseihis/bnnprint/blob/main/notebooks/qkerasbnntuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install keras-tuner QKeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyparser: filename=pyparser-1.0-py3-none-any.whl size=4941 sha256=6560997c488a8d35871ac37902d90653d90d5a3c0539efc9daaa05d1feec2030
  Stored in directory: /root/.cache/pip/wheels/3f/a4/1a/fdbe78211760c69a9c7635d224d3683c8ed2519e01f53a81e0
  Created wheel for parse: filename=parse-1.6.5-py3-none-any.whl size=18174 sha256=23e12d9fa3447506a33186794990b0e9ce2aac04c9984ad935dfc62f8c2f2e06
  Stored in directory: /root/.cache/pip/wheels/37/ef/ff/5004e21f680c6489785b14199d1716e5db6

In [ ]:
def get_csv_filenames():
    filenames = os.listdir()
    csv_filenames = [filename for filename in filenames if filename.endswith(".csv")]
    return csv_filenames

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner import HyperParameters
import keras_tuner as kt
from tensorflow.keras.utils import to_categorical

import numpy as np
import pandas as pd
import os

# data and metric imports
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

from qkeras import *
import qkeras.utils as qutils


def build_model(input_shape, output_shape, hp):
    model = keras.Sequential()
    model.add(QActivation("quantized_bits(4, keep_negative=0)"))
    model.add(QDense(units=40,kernel_quantizer="binary", use_bias=False, input_shape=(input_shape,)))
    #model.add(layers.BatchNormalization(scale=False))
    #model.add(layers.Activation('relu'))
    model.add(QActivation("binary"))
    #model.add(layers.Dense(units=output_shape))
    #model.add(layers.Dropout(rate=hp.Float("dropout", min_value=0, max_value=0.4)))
    model.add(QDense(units=output_shape, kernel_quantizer="binary(alpha=1)", use_bias=False))
    model.add(layers.Activation('softmax'))

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float("learning_rate", min_value=0.001, max_value=0.02)),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model


def train_csv(fname,tri=20,ep=50):
    df = pd.read_csv(fname, header=None)
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    X = pd.DataFrame(X.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))))
    X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(X, y, test_size=0.3, random_state=42)
    (_, input_shape) = X.shape
    output_shape = y.nunique()
    y_train = to_categorical(y_train, num_classes=output_shape)
    y_test = to_categorical(y_test, num_classes=output_shape)

    def my_builder(hp):
        return build_model(input_shape, output_shape, hp)

    tuna = kt.BayesianOptimization(
        hypermodel=my_builder,
        objective="val_accuracy",
        max_trials=tri,
        project_name="proj"+fname
    )
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    tuna.search(X_train, y_train, epochs=ep, validation_data=(X_test, y_test), callbacks=[early_stop])
    print(fname+"summary")
    #tuna.results_summary()
    model = tuna.get_best_models(num_models=1)[0]
    x_hack = tf.random.normal(shape=(1,input_shape),dtype=tf.float32)
    output=model.predict(x_hack)
    qutils.model_save_quantized_weights(model,'weights'+fname+'.h5')
    model.save(fname+"fptuner.h5")
    model.evaluate(X_test, y_test)

In [ ]:
os.chdir('drive/MyDrive/colab notebooks/dataset')

In [ ]:
train_csv("pendigits.csv")

Trial 20 Complete [00h 00m 15s]
val_accuracy: 0.8012449741363525

Best val_accuracy So Far: 0.8683859705924988
Total elapsed time: 00h 05m 43s
pendigits.csvsummary


1/1 [==============================] - 0s 360ms/step
... quantizing model
71/71 [==============================] - 0s 3ms/step - loss: 0.5971 - accuracy: 0.8684


In [ ]:
for fname in get_csv_filenames():
  train_csv(fname)

Har.csvsummary
1/1 [==============================] - 0s 373ms/step
... quantizing model
  batch_normalization has not been quantized
28/28 [==============================] - 0s 3ms/step - loss: 2.9639 - accuracy: 0.4859
winequality-red.csvsummary


1/1 [==============================] - 0s 372ms/step
... quantizing model
  batch_normalization has not been quantized
15/15 [==============================] - 0s 4ms/step - loss: 2.6770 - accuracy: 0.5542
arrhythmia.csvsummary


1/1 [==============================] - 0s 373ms/step
... quantizing model
  batch_normalization has not been quantized
5/5 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.5926
cardio.csvsummary


1/1 [==============================] - 0s 372ms/step
... quantizing model
  batch_normalization has not been quantized
20/20 [==============================] - 0s 4ms/step - loss: 0.5799 - accuracy: 0.8746
winequality-white.csvsummary


1/1 [==============================] - 1s 558ms/step
... quantizing model
  batch_normalization has not been quantized
46/46 [==============================] - 1s 5ms/step - loss: 2.9946 - accuracy: 0.4844


gasId.csvsummary
1/1 [==============================] - 0s 395ms/step
... quantizing model
  batch_normalization has not been quantized
131/131 [==============================] - 1s 3ms/step - loss: 1.1046 - accuracy: 0.7263
pendigits.csvsummary


1/1 [==============================] - 0s 374ms/step
... quantizing model
  batch_normalization has not been quantized
71/71 [==============================] - 0s 3ms/step - loss: 0.5642 - accuracy: 0.8635


IsADirectoryError: ignored

In [ ]:
df = pd.read_csv('winequality_white.csv', header=None)
Xx = df.iloc[:, :-1]
y = df.iloc[:, -1]
X = pd.DataFrame(Xx.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))))

In [ ]:
%pip install h5py


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from keras.models import load_model

# Load the model from the HDF5 file
model = load_model('winequality_white.csvfptuner.h5', custom_objects={'QActivation': QActivation,'QDense':QDense})

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [ ]:
#x_hack = tf.random.normal(shape=(1,16),dtype=tf.float32)
#x_hack = tf.abs(x_hack)
#output=model.predict(x_hack)
#inpux = np.expand_dims(X.iloc[0], axis=0)
inpux = X.iloc[0:5]
#inpux = np.array([[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]])
lo=get_layer_outputs(model,inpux)
yoyo = lo[2]
gol = lo[4]
lo

[array([[0.3125, 0.1875, 0.1875, 0.3125, 0.125 , 0.125 , 0.375 , 0.25  ,
         0.25  , 0.25  , 0.125 ],
        [0.25  , 0.1875, 0.1875, 0.    , 0.125 , 0.0625, 0.3125, 0.125 ,
         0.5   , 0.3125, 0.25  ],
        [0.4375, 0.1875, 0.25  , 0.125 , 0.125 , 0.125 , 0.1875, 0.125 ,
         0.5   , 0.25  , 0.3125],
        [0.3125, 0.125 , 0.1875, 0.125 , 0.125 , 0.1875, 0.4375, 0.1875,
         0.4375, 0.1875, 0.3125],
        [0.3125, 0.125 , 0.1875, 0.125 , 0.125 , 0.1875, 0.4375, 0.1875,
         0.4375, 0.1875, 0.3125]], dtype=float32),
 array([[ 0.5      ,  0.5      ,  0.       , -0.25     ,  0.       ,
         -0.28125  , -0.1875   ,  0.21875  ,  0.       , -0.1875   ,
          0.0625   , -0.03125  ,  0.       ,  0.15625  ,  0.0625   ,
         -0.125    ,  0.6875   ,  0.       ,  1.125    , -0.1875   ,
         -0.28125  , -0.03125  , -0.0625   ,  0.       , -0.8125   ,
         -0.15625  ,  0.       ,  0.0625   ,  0.125    ,  0.21875  ,
          0.5      , -0.125    ,  

In [ ]:
(lo[3]+40)/2

array([[21., 20., 25., 23., 23., 21., 15.],
       [24., 23., 28., 28., 24., 24., 18.],
       [20., 21., 24., 26., 20., 20., 14.],
       [20., 21., 26., 26., 22., 24., 18.],
       [20., 21., 26., 26., 22., 24., 18.]], dtype=float32)

In [ ]:
yozo = np.maximum(yoyo[1],0)
asbin=yozo.astype(np.uint8).tobytes()
bs=binascii.hexlify(asbin).decode()[1::2]
bs[::-1]

'0000110101101100100001110111100010010011'

In [ ]:
xrisos = np.argmax(gol,axis=1)
qinp = lo[0]

In [ ]:
xrisos

array([5])

In [ ]:
import h5py

filename = "weightspendigits.csv.h5"

h5 = h5py.File(filename,'r')

h5.visit(print)
#print(h5['q_dense/q_dense/kernel:0'][:,:])
#print(h5['q_batch_normalization/q_batch_normalization/beta:0'][:])
sw0 = h5['q_dense/q_dense/kernel:0'][:,:]
sw1 = h5['q_dense_1/q_dense_1/kernel:0'][:,:]

h5.close()

activation
q_activation
q_activation_1
q_dense
q_dense/q_dense
q_dense/q_dense/kernel:0
q_dense_1
q_dense_1/q_dense_1
q_dense_1/q_dense_1/kernel:0
top_level_model_weights


In [ ]:
def mysign(a):
  signs = np.sign(a)

  # Replace the sign of the zeroes with 1 (positive) or -1 (negative)
  signs_no_zero = np.where(signs == 0, 1, signs)
  return signs_no_zero

In [ ]:
def grup(x):
  x1 = (x) @ mysign(sw0)
  x2 = mysign(x1)
  x3 = x2 @ sw1
  return (x1,x2,x3)
def drut(x):
  x1 = x @ mysign(sw0)
  x2 = mysign(x1)
  x3 = x2 @ sw1
  am = np.argmax(x3,axis=1)
  return am

In [ ]:
mysign(sw0)

array([[-1., -1., -1.,  1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.,  1.,  1.,
         1.],
       [ 1.,  1., -1., -1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1.,
         1.,  1., -1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         1.,  1., -1., -1.,  1.,  1.,  1., -1., -1., -1., -1.,  1., -1.,
         1.],
       [-1., -1., -1.,  1., -1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,
         1., -1.,  1., -1., -1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,
        -1., -1., -1.,  1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1.,
        -1.],
       [-1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1.,
        -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1., -1., -1.,  1.,
         1., -1.,  1., -1.,  1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,
        -1.],
       [-1.,  1.,  1., -1.,  1., -1.,  1.,  1., -1., -1.,  1.,  1., 

In [ ]:
inm = lo[0]*16
asw0 = mysign(sw0)
psw0 = np.maximum(asw0,0)
nsw0 = np.minimum(asw0,0)
hmp = inm @ psw0
hmn = inm @ nsw0
inm,hmp, hmn, hmp-hmn, grup(inm)

(array([[ 8., 15.,  4., 13.,  9.,  6.,  4.,  0.,  0.,  4.,  9.,  8., 15.,
         14.,  6., 15.]], dtype=float32),
 array([[45., 60., 40., 63., 50., 43., 79., 37., 46., 49., 67., 68., 78.,
         80., 86., 37., 63., 55., 75., 74., 54., 68., 74., 79., 73., 68.,
         72., 90., 65., 28., 84., 32., 76., 64., 40., 79., 55., 71., 91.,
         66.]], dtype=float32),
 array([[ -85.,  -70.,  -90.,  -67.,  -80.,  -87.,  -51.,  -93.,  -84.,
          -81.,  -63.,  -62.,  -52.,  -50.,  -44.,  -93.,  -67.,  -75.,
          -55.,  -56.,  -76.,  -62.,  -56.,  -51.,  -57.,  -62.,  -58.,
          -40.,  -65., -102.,  -46.,  -98.,  -54.,  -66.,  -90.,  -51.,
          -75.,  -59.,  -39.,  -64.]], dtype=float32),
 array([[130., 130., 130., 130., 130., 130., 130., 130., 130., 130., 130.,
         130., 130., 130., 130., 130., 130., 130., 130., 130., 130., 130.,
         130., 130., 130., 130., 130., 130., 130., 130., 130., 130., 130.,
         130., 130., 130., 130., 130., 130., 130.]], dtype=flo

In [ ]:
15*16

240

In [ ]:
drut(qinp)[0:20]

array([6, 2, 1, 4, 7, 6, 4, 0, 5, 0, 9, 8, 5, 9, 7, 3, 3, 9, 2, 2])

In [ ]:
y[0:10]

0    8
1    2
2    1
3    4
4    1
5    6
6    4
7    0
8    5
9    0
Name: 16, dtype: int64

In [ ]:
glo = lo[2][175]
gor = grup(qinp[175])[1]
glo, gor, np.where(np.not_equal(glo,gor))

(array([-1., -1., -1., -1., -1., -1., -1., -1.,  1.,  1.,  1., -1.,  1.,
        -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
         1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1., -1., -1.,
        -1.], dtype=float32),
 array([-1., -1., -1., -1., -1., -1., -1., -1.,  1.,  0.,  0., -1.,  1.,
        -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
         1.,  1., -1., -1., -1.,  1.,  0., -1.,  1., -1., -1., -1., -1.,
        -1.], dtype=float32),
 (array([ 9, 10, 32]),))

In [ ]:
lo[0]

array([[0.5   , 0.9375, 0.25  , ..., 0.875 , 0.375 , 0.9375],
       [0.    , 0.875 , 0.25  , ..., 0.    , 0.9375, 0.0625],
       [0.    , 0.5625, 0.3125, ..., 0.25  , 0.1875, 0.    ],
       ...,
       [0.3125, 0.5   , 0.5625, ..., 0.    , 0.9375, 0.0625],
       [0.875 , 0.875 , 0.375 , ..., 0.    , 0.    , 0.0625],
       [0.6875, 0.9375, 0.375 , ..., 0.25  , 0.5625, 0.    ]],
      dtype=float32)

In [ ]:
import binascii
asd=(lo[0][0]*16).astype(np.uint8).tobytes()
binascii.hexlify(asd).decode()[1::2]

'53352264442'

In [ ]:
binn = np.maximum(lo[2],0)
asbin=binn.astype(np.uint8).tobytes()
binascii.hexlify(asbin).decode()[1::2]

'0000001000111110001101111111101010010111'

In [ ]:
matrix_bin_string(sw0)

'0001000011111110110111111111000000010111110001101101011010000000001100111000010100010010101011010001010101000100000101000101101111100011110100100110101011001010011010110011010011110001111010111111001000111101100011110001000111010100011100011011110000011001011010010000101111111011001110000000100100101101110001001111101011101011010000010010111010011111011011001001110001111001101111011110010101010011010100110100010000001110111110000000101011001100001110001010110010110110010100010011000000001010011110110101100000110111100000100001111000100111100100101100111001101100001100011110110000100110111100010000001000101110001101111001101010011110'

In [ ]:
matrix_bin_string(sw1.T)

'1000001001011000001111101001101100001110001111001010000101111101010001111111000101111100001100010101000001100011101000110111111101010100011110011110101101101010110100000100000110101110001101100001000100001011111111100000010110110100110100100001000011110110000000101111000000001110011011100010000111100001101000101011101000000000000011110011011101011110110111100000110001101000011011011111010000010000'

In [ ]:
def matrix_bin_string(mat):
  flatw=mysign(mat).flatten()
  bflat = np.maximum(flatw,0)
  flbin=bflat.astype(np.uint8).tobytes()
  return binascii.hexlify(flbin).decode()[1::2]

In [ ]:
sw1

array([[ 1., -1., -1., -1.,  1., -1., -1., -1., -1., -1.],
       [-1., -1.,  1.,  1.,  1., -1., -1.,  1., -1., -1.],
       [-1.,  1.,  1.,  1., -1., -1., -1.,  1., -1., -1.],
       [-1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1., -1.],
       [-1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.],
       [-1.,  1.,  1.,  1., -1., -1., -1.,  1., -1.,  1.],
       [ 1., -1., -1.,  1., -1.,  1., -1.,  1., -1., -1.],
       [-1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1.],
       [-1.,  1., -1., -1., -1.,  1.,  1., -1., -1., -1.],
       [ 1., -1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1.],
       [-1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1.,  1.],
       [ 1., -1.,  1.,  1., -1.,  1.,  1., -1., -1., -1.],
       [ 1., -1., -1., -1., -1.,  1., -1., -1.,  1.,  1.],
       [-1., -1., -1.,  1., -1.,  1.,  1., -1.,  1., -1.],
       [-1., -1., -1., -1., -1.,  1.,  1., -1.,  1., -1.],
       [-1.,  1.,  1., -1.,  1., -1., -1.,  1.,  1., -1.],
       [-1., -1., -1., -1.,  1., -1., -1.,  1., -1., -1.

In [ ]:
def boopify(x):
  return (x+40)/2

In [ ]:
boopify(lo[3])

array([[26., 16., 18., 17., 18., 26., 25., 19., 30., 22.]], dtype=float32)

In [ ]:
np.argmax(lo[3])

8

In [ ]:
#dut = yoyo @ (np.sign(swax))
#dd , gg = np.argmax(dut,axis=1),np.argmax(gol,axis=1)
hm=np.where(np.not_equal(drut(qinp),y[0:199]))
ex=np.where(np.not_equal(xrisos,y[0:199]))
#(dd[hm],gg[hm],hm)
hm,ex,np.where(np.not_equal(hm,ex))

((array([  9,  19,  27,  29,  38,  49,  61,  63,  66,  70,  97,  99, 117,
         126, 132, 134, 154, 160, 167, 180, 194, 196]),),
 (array([  9,  19,  27,  29,  38,  49,  61,  63,  66,  70,  97,  99, 117,
         126, 132, 134, 154, 160, 167, 180, 194, 196]),),
 (array([], dtype=int64), array([], dtype=int64)))

In [ ]:
h5.keys()

ValueError: ignored

In [ ]:
h5['q_dense_1/q_dense_1/kernel:0'][:,:]

ValueError: ignored

In [ ]:
list(QBatchNormalization().quantizers)

In [ ]:
import keras.backend as K

def get_layer_outputs(model, x):
    """
    Returns the output of every layer in a Keras model evaluated at the input x.
    
    Args:
    model: A Keras model object.
    x: The input to the model as a Numpy array or a list of Numpy arrays.
    
    Returns:
    A list of Numpy arrays, where the i-th element is the output of the i-th layer
    in the model evaluated at the input x.
    """
    # Create a function that computes the output of every layer given the input
    layer_outputs = [layer.output for layer in model.layers]
    layer_eval_fn = K.function(inputs=[model.input], outputs=layer_outputs)
    
    # Evaluate the function at the input
    output_values = layer_eval_fn([x])
    
    return output_values


In [ ]:
import binascii
asd=(lo[0]*16).astype(int)
asduts = np.round((inpux*16)).astype(int)
asdut = np.minimum(15,asduts)

np.array_equal(asd,asdut)

True

In [ ]:
import binascii
def qnorm(Xa):
  return pd.DataFrame(Xa.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))))

def qfour(Xa):
  Xc = qnorm(Xa)
  Xb = np.round(Xc*16).astype(int)
  return np.minimum(15,Xb)

def hexdump4(X):
  N = X.astype(np.uint8).tobytes()
  return binascii.hexlify(N).decode()[1::2]

def hexiby(X):
  Y = [row.to_numpy() for index, row in qfour(X).iterrows()]
  return [hexdump4(x) for x in Y]


In [ ]:
jkj = hexiby(Xx)[0:20]
#np.array_equal(asd,jkj)
jkj

['8f4d96400498fe6f',
 '0e4f7c572260b0f1',
 '095bceffcc884430',
 '0f1f1b37e5f7c4b0',
 '0b8dffddaa665380',
 'ffef8c380360c333',
 '0f0c46e6fbcfa8b0',
 '060a21a0f7ef6f09',
 '2e28c690130a5fff',
 '9f4c0540c2f8ce4e',
 'ce5f0baafdf6d030',
 '8faae43035dbff0e',
 'ffcf6c59d8a23000',
 'fc9f0e49dbf6d130',
 '0e6fdee8d29084f4',
 '6c9fffbbd6d15003',
 '8dbfcc88f7f28001',
 'fdaf4cdbfaf49003',
 '4b7f9a54003290f0',
 '0c3fcfbc581370f0']

In [ ]:
from keras.models import load_model
def get_results(fname):
    df = pd.read_csv(fname, header=None)
    X = df.iloc[:, :-1]
    
    X = pd.DataFrame(X.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))))
    
    model = load_model(fname+"fptuner.h5", custom_objects={'QActivation': QActivation,'QDense':QDense})
   
    output=model.predict(X)
    clases = np.argmax(output, axis=1)
    return clases
    

In [ ]:
get_results("pendigits.csv")[0:14]

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


235/235 [==============================] - 1s 2ms/step


array([8, 2, 1, 4, 1, 6, 4, 0, 5, 0, 9, 8, 5, 9])

In [ ]:
def write_strings_to_file(strings, file_path):
    with open(file_path, 'w') as file:
        for string in strings:
            file.write(str(string) + '\n')

In [ ]:
for fname in get_csv_filenames():
  gres = get_results(fname).tolist()
  write_strings_to_file(gres,'results_'+fname+'.results')

 1/50 [..............................] - ETA: 5s

/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


 1/67 [..............................] - ETA: 7s

/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


  1/154 [..............................] - ETA: 17s

...

 1/93 [..............................] - ETA: 9s

/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


93/93 [==============================] - 0s 2ms/step


/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


235/235 [==============================] - 1s 2ms/step
